In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2
import sys
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
import json
import jsonlines
sys.path.append('/Users/tyler/Documents/programming/africa_nlp/text-gnn')
from src.shared import utils

## Setup directories

In [2]:
project_dir = '/Users/tyler/Documents/programming/africa_nlp/text-gnn'

## Helsinki dataset

### Documents

In [3]:
data_dir = f'{project_dir}/results/hel_data'

In [4]:
df = pd.read_csv(f'{data_dir}/dataset.csv',sep=';')

In [5]:
n_classes = df.document_type.nunique()
print(f'{len(df)} total documents with {n_classes} classes')

457 total documents with 3 classes


In [6]:
df.document_type.value_counts(dropna=False)

news     221
bunge    199
books     37
Name: document_type, dtype: int64

In [7]:
df['n_words'] = df.document_content.apply(lambda x:len(x.split()))

In [8]:
mean_words = round(df.n_words.mean(),1)
median_words = round(df.n_words.median(),1)

In [9]:
print(f'Mean words per document of {mean_words}')

Mean words per document of 52887.8


In [10]:
print(f'Median words per document of {median_words}')

Median words per document of 38732.0


In [11]:
total_words = df.n_words.sum()
print(f'{total_words/int(1e6):.2f} million words total')

In [13]:
## File size is 160.1 MB

### Vocab

In [14]:
count_threshold = 2

In [15]:
stemming_dir = f'{project_dir}/results/hel_results/stemming'

In [16]:
path = f'{stemming_dir}/vocab_counts.json'
with open(path,'r') as f:
    vocab_counts = json.load(f)

In [17]:
words_above_threshold = [word for word, count in vocab_counts.items() if count >= count_threshold]

In [18]:
print(f'{len(vocab_counts)} words in vocab')

434449 words in vocab


In [19]:
print(f'{len(words_above_threshold)} words in with at least {count_threshold} occurences in corpus')

212920 words in with at least 2 occurences in corpus


In [20]:
path = f'{stemming_dir}/stemming_cleaned.json'
with open(path,'r') as f:
    stemming_map = json.load(f)

In [21]:
stemmed_words = [stemming_map[word] for word in words_above_threshold]

In [22]:
print(f'{len(set(stemmed_words))} words in vocab after applying stemming')

71020 words in vocab after applying stemming


## Zenodo dataset

### Documents

In [23]:
data_dir = f'{project_dir}/results/zen_data'

In [24]:
df = pd.read_csv(f'{data_dir}/dataset.csv',sep=';')

In [25]:
n_classes = df.document_type.nunique()
print(f'{len(df)} total documents with {n_classes} classes')

23268 total documents with 6 classes


In [26]:
df.document_type.value_counts(dropna=False)

kitaifa      10242
michezo       6004
burudani      2229
uchumi        2028
kimataifa     1906
afya           859
Name: document_type, dtype: int64

In [27]:
df['n_words'] = df.document_content.apply(lambda x:len(x.split()))

In [28]:
mean_words = round(df.n_words.mean(),1)
median_words = round(df.n_words.median(),1)

In [29]:
print(f'Mean words per document of {mean_words}')

Mean words per document of 332.5


In [30]:
print(f'Median words per document of {median_words}')

Median words per document of 276.0


In [31]:
total_words = df.n_words.sum()
print(f'{total_words/int(1e6):.2f} million words total')

In [33]:
## File size is 52.3 MB

### Vocab

In [34]:
stemming_dir = f'{project_dir}/results/zen_results/stemming'

In [35]:
path = f'{stemming_dir}/vocab_counts.json'
with open(path,'r') as f:
    vocab_counts = json.load(f)

In [36]:
words_above_threshold = [word for word, count in vocab_counts.items() if count >= count_threshold]

In [37]:
print(f'{len(vocab_counts)} words in vocab')

198556 words in vocab


In [38]:
print(f'{len(words_above_threshold)} words in with at least {count_threshold} occurences in corpus')

95987 words in with at least 2 occurences in corpus


In [39]:
path = f'{stemming_dir}/stemming_cleaned.json'
with open(path,'r') as f:
    stemming_map = json.load(f)

In [40]:
stemmed_words = [stemming_map[word] for word in words_above_threshold]

In [41]:
print(f'{len(set(stemmed_words))} words in vocab after applying stemming')

39872 words in vocab after applying stemming
